In [ ]:
from agent import Agent
import json

# Load agent definition from the JSON file
import pathlib

agent_def_path = pathlib.Path("agent_definition.json")
if not agent_def_path.exists():
    raise FileNotFoundError(f"Agent definition file not found: {agent_def_path.resolve()}")

with agent_def_path.open("r", encoding="utf-8") as f:
    agent_definition = json.load(f)

# # Show the loaded definition
# agent_definition
ff_agent = await Agent.create(agent_definition)
result = await ff_agent.run_agent("what is the list of all players on on Blaine's fantasy football team?")

2025-08-21 14:52:14,995 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:14,996 - mcp.client.streamable_http - INFO - Received session ID: 4102b97346c24bc59f24dde6812548e5
2025-08-21 14:52:15,001 - mcp.client.streamable_http - INFO - Negotiated protocol version: 2025-06-18
2025-08-21 14:52:15,024 - httpx - INFO - HTTP Request: GET http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:15,027 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 202 Accepted"
2025-08-21 14:52:15,038 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:15,052 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:15,059 - root - INFO - Successfully connected to MCP server: ff_tools (http)
2025-08-21 14:52:15,631 - httpx - INFO - HTTP Request: GET https://sql-mcp-demo.azurewebsites.us/sse "HTTP/1.1 200 OK"
2025-08-21 14:52:15


--- Agent Thoughts ---
User asks: "what who is on Blaine's team?" Likely want roster. We need to call ff_tools_get_roster with team_name "Blaine".

2025-08-21 14:52:27,034 - semantic_kernel.connectors.ai.chat_completion_client_base - INFO - processing 1 tool calls in parallel.
2025-08-21 14:52:27,035 - semantic_kernel.kernel - INFO - Calling ff_tools-get_roster function with args: {'team_name': 'Blaine'}
2025-08-21 14:52:27,037 - semantic_kernel.functions.kernel_function - INFO - Function ff_tools-get_roster invoking.
2025-08-21 14:52:27,044 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:27,163 - semantic_kernel.functions.kernel_function - INFO - Function ff_tools-get_roster succeeded.
2025-08-21 14:52:27,164 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.126242s



--- Agent Tools ---
Tool: ff_tools-get_roster
Arguments: {'team_name': 'Blaine'}

--- Agent Message ---


2025-08-21 14:52:27,439 - httpx - INFO - HTTP Request: POST http://ollama.home/api/chat "HTTP/1.1 200 OK"


...


--- Agent Thoughts ---
We need to answer: "what who is on Blaine's team?" We need to list roster. We got error with get_roster for team Blaine. But we have list of teams. "In GPT We Trust" has short "Blaine". So Blaine's team is "In GPT We Trust". We need roster.

We must fetch roster via get_roster with team_name "In GPT We Trust". Let's do that.

2025-08-21 14:52:30,240 - semantic_kernel.connectors.ai.chat_completion_client_base - INFO - processing 1 tool calls in parallel.
2025-08-21 14:52:30,242 - semantic_kernel.kernel - INFO - Calling get_roster function with args: {'team_name': 'In GPT We Trust'}
2025-08-21 14:52:30,243 - semantic_kernel.functions.kernel_function - INFO - Function ff_tools-get_roster invoking.
2025-08-21 14:52:30,257 - httpx - INFO - HTTP Request: POST http://192.168.86.103:8000/mcp "HTTP/1.1 200 OK"
2025-08-21 14:52:30,269 - semantic_kernel.functions.kernel_function - INFO - Function ff_tools-get_roster succeeded.
2025-08-21 14:52:30,271 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 0.027044s



--- Agent Tools ---
Tool: get_roster
Arguments: {'team_name': 'In GPT We Trust'}

--- Agent Message ---


2025-08-21 14:52:35,055 - httpx - INFO - HTTP Request: POST http://ollama.home/api/chat "HTTP/1.1 200 OK"



--- Agent Thoughts ---
We have the JSON output from the function. Need to extract all names that meet:

- injured: false (not injured)
- bench: false (bench: false indicates not on bench, meaning active)
- active: true (roster_status: "Active")

Also ensure not in "Rookie" roster_status? The conditions say bench false and active true, so we need roster_status "Active". Not "Bench". Also "bench" field refers to the bench attribute for quarterbacks? Many entries have bench status "Bench" or "Active". We need bench: false. That means the field bench is false. But we need active: true. Actually we need bench: false and active: true. That is contradictory: "bench: false" means not on bench. That implies active. So maybe we interpret bench: false means that the "bench" field is false. But the "roster_status" field indicates active. The "bench" field is present for some entries like "Joe Milton" has bench: "Bench". But the field bench is not boolean; it's a string. But in some entries like "

In [7]:
result.get("thoughts")

'User asks: "what who is on Blaine\'s team?" Likely want roster. We need to call ff_tools_get_roster with team_name "Blaine".We attempted get_roster with team_name "Blaine" but error: maybe wrong name? Need correct team name. Maybe "Blaine\'s"? The user said Blaine\'s team. Let\'s list teams.We need to answer: "what who is on Blaine\'s team?" We need to list roster. We got error with get_roster for team Blaine. But we have list of teams. "In GPT We Trust" has short "Blaine". So Blaine\'s team is "In GPT We Trust". We need roster.\n\nWe must fetch roster via get_roster with team_name "In GPT We Trust". Let\'s do that.We have the JSON output from the function. Need to extract all names that meet:\n\n- injured: false (not injured)\n- bench: false (bench: false indicates not on bench, meaning active)\n- active: true (roster_status: "Active")\n\nAlso ensure not in "Rookie" roster_status? The conditions say bench false and active true, so we need roster_status "Active". Not "Bench". Also "be

In [8]:
result.get("tool_calls")

[{'ff_tools-get_roster': {'team_name': 'Blaine'}},
 {'get_teams': {}},
 {'get_roster': {'team_name': 'In GPT We Trust'}}]

In [9]:
result.get("messages")

"The active, non‑injured players from the list are:\n\n- Wan'Dale Robinson  \n- Damian Thompson  \n- Damien Thompson"

In [10]:
# await ff_agent.run_agent("what is in my db??")